In [22]:
from Project import collector
from Project.SimulatedAgent.AgentEnums import AgentRole

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

number_of_steps = 300
number_of_simulations = 100
number_of_agents = 5
width = 30
height = 30
with_comm = False

In [13]:
%%capture
data = collector.single_run_collector(map_width=width, map_height=height, number_of_simulation_steps=number_of_steps, number_of_agents=number_of_agents ,with_communication=with_comm, seed=123)
data.head()


step: 0

step: 1

step: 2

step: 3

step: 4

step: 5

step: 6

step: 7

step: 8

step: 9

step: 10

step: 11

step: 12

step: 13

step: 14

step: 15

step: 16

step: 17

step: 18

step: 19

step: 20

step: 21

step: 22

step: 23

step: 24

step: 25

step: 26

step: 27

step: 28

step: 29

step: 30

step: 31

step: 32

step: 33

step: 34

step: 35

step: 36

step: 37

step: 38

step: 39

step: 40

step: 41

step: 42

step: 43

step: 44

step: 45

step: 46

step: 47

step: 48

step: 49

step: 50

step: 51

step: 52

step: 53

step: 54

step: 55

step: 56

step: 57

step: 58

step: 59

step: 60

step: 61

step: 62

step: 63

step: 64

step: 65

step: 66

step: 67

step: 68

step: 69

step: 70

step: 71

step: 72

step: 73

step: 74

step: 75

step: 76

step: 77

step: 78

step: 79

step: 80

step: 81

step: 82

step: 83

step: 84

step: 85

step: 86

step: 87

step: 88

step: 89

step: 90

step: 91

step: 92

step: 93

step: 94

step: 95

step: 96

step: 97

step: 98

step: 99

step: 100

,Agent,Role,Round,Gold,Killed_Wumpus,Map_Progress
0,0,HUNTER,1,0,0,2
1,1,KNIGHT,1,0,0,1
2,2,CARTOGRAPHER,1,0,0,2
3,3,KNIGHT,1,0,0,2
4,4,KNIGHT,1,0,0,2


In [32]:
# Plot progress single run
plt = make_subplots(rows=2, cols=2)
d = {}
goals = ["Gold", "Killed_Wumpus", "Map_Progress"]
for goalID in range(len(goals)):
    for agent in range(0, number_of_agents):
        df = data[data["Agent"] == agent]
        d["x"] = df["Round"].values
        d["y"] = df[goals[goalID]].values
        txt = [f"{agent}: {df["Role"].values[0]}" for _ in range(0, number_of_steps)]
        plt.add_trace(go.Line(
         d, text= txt, showlegend = False
        ), row=(goalID//2)+1, col=goalID%2+1)
    plt.update_xaxes(title_text="Round", row=(goalID//2)+1, col=goalID%2+1)
    plt.update_yaxes(title_text=goals[goalID], row=(goalID//2)+1, col=goalID%2+1)

plt

C:\Users\Luza3\PycharmProjects\Wumpus\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [11]:
%%capture
data, ss = collector.multiple_run_result_collector(number_of_simulations , map_width=width, map_height=height, number_of_simulation_steps=number_of_steps, number_of_agents=number_of_agents ,with_communication=with_comm, start_seed=140)

KeyboardInterrupt: 

In [10]:
%%capture cap --no-stderr
def compress_data_for_number_of_simulation(df: pd.DataFrame, simulation: int):
    tempData = []
    columns=["Role", "Gold", "Killed_Wumpus", "Map_Progress"]
    for i in range(1, simulation):
        for role in AgentRole:
            temp = df.where(df["Role"] == role.name).where(df["Simulation"] == i)
            temp = temp[temp["Agent"].notna()]
            num = len(temp.index)
            row = [0 for _ in range(len(columns))]
            if num != 0:
                tempData.append([role.name, temp.loc[:,"Gold"].mean(), temp.loc[:,"Killed_Wumpus"].mean(), temp.loc[:,"Map_Progress"].mean()])

    df = pd.DataFrame(tempData, columns=columns)
    columns=["Role", "Gold", "Killed_Wumpus", "Map_Progress"]
    tempData = []
    for role in AgentRole:
        tempData.append([role.name, df[df["Role"] == role.name].loc[:,"Gold"].mean(), df[df["Role"] == role.name].loc[:,"Killed_Wumpus"].mean(),df[df["Role"] == role.name].loc[:,"Map_Progress"].mean()])
    return pd.DataFrame(tempData, columns=columns)
        
data = compress_data_for_number_of_simulation(data, number_of_simulations)
ss

KeyError: 'Simulation'

In [6]:
# Plot progress single run
plt = make_subplots(rows=1, cols=3, horizontal_spacing=0.1)
goals = ["Gold", "Killed_Wumpus", "Map_Progress"]

for goalID in range(len(goals)):
    plt.add_trace(go.Bar(x=data['Role'].to_list(), y=data[goals[goalID]].to_list()
                         , showlegend = False)
                         , row=(goalID//3)+1, col=goalID%3+1)
    plt.update_xaxes(title_text="", row=(goalID//3)+1, col=goalID%3+1)
    plt.update_yaxes(title_text=goals[goalID], row=(goalID//3)+1, col=goalID%3+1)

plt

In [68]:
%%capture
# avg Verlauf
def compress_data_for_number_of_steps(df: pd.DataFrame, steps: int):
    tempData = []
    columns=["Round","Role", "Gold", "Killed_Wumpus", "Map_Progress"]
    for i in range(0, steps):
        for role in AgentRole:
            temp = df.where(df["Role"] == role.name).where(df["Round"] == i)
            temp = temp[temp["Agent"].notna()]
            num = len(temp.index)
            if num != 0:
                tempData.append([i, role.name, temp.loc[:,"Gold"].mean(), temp.loc[:,"Killed_Wumpus"].mean(), temp.loc[:,"Map_Progress"].mean()])

    return pd.DataFrame(tempData, columns=columns)

def add_data_points(df1: pd.DataFrame, df2: pd.DataFrame, steps: int):
    tempData = []
    columns=["Round","Role", "Gold", "Killed_Wumpus", "Map_Progress"]
    for i in range(0, steps):
        for role in AgentRole:
            temp1 = df1.where(df1["Role"] == role.name).where(df1["Round"] == i)
            temp1 = temp1[temp1["Role"].notna()]
            temp2 = df2.where(df2["Role"] == role.name).where(df2["Round"] == i)
            temp2 = temp2[temp2["Role"].notna()]
            num = max(len(temp1.index), len(temp2.index))
            if num != 0:
                gold = 0
                map = 0
                wumpus = 0
                try:
                    gold += float(temp1.loc[:,"Gold"].iloc[0])
                except:
                    pass
                try:
                    gold += float(temp2.loc[:,"Gold"].iloc[0])
                except:
                    pass
                try:
                    wumpus += float(temp1.loc[:,"Killed_Wumpus"].iloc[0])
                except:
                    pass
                try:
                    wumpus += float(temp2.loc[:,"Killed_Wumpus"].iloc[0])
                except:
                    pass
                try:
                    map += float(temp1.loc[:,"Map_Progress"].iloc[0])
                except:
                    pass
                try:
                    map += float(temp2.loc[:,"Map_Progress"].iloc[0])
                except:
                    pass
                tempData.append([i, role.name, gold, wumpus, map])
    return pd.DataFrame(tempData, columns=columns)

n = 1
data = compress_data_for_number_of_steps(collector.single_run_collector(map_width=width, map_height=height, number_of_simulation_steps=number_of_steps, number_of_agents=number_of_agents ,with_communication=with_comm, seed=123), number_of_steps)
for delta in range(1, number_of_simulations):
    temp = compress_data_for_number_of_steps(collector.single_run_collector(map_width=width, map_height=height, number_of_simulation_steps=number_of_steps, number_of_agents=number_of_agents ,with_communication=with_comm, seed=123 + delta), number_of_steps)
    data = add_data_points(data, temp, number_of_steps)

#avg values
for id in range(len(data.values)):
    temp = data.values[id].flatten().tolist()
    temp[2] = temp[2] / number_of_simulations
    temp[3] = temp[3] / number_of_simulations
    temp[4] = temp[4] / number_of_simulations
    data.iloc[id] = temp

data


step: 0

step: 1

step: 2

step: 3

step: 4

step: 5

step: 6

step: 7

step: 8

step: 9

step: 10

step: 11

step: 12

step: 13

step: 14

step: 15

step: 16

step: 17

step: 18

step: 19

step: 20

step: 21

step: 22

step: 23

step: 24

step: 25

step: 26

step: 27

step: 28

step: 29

step: 30

step: 31

step: 32

step: 33

step: 34

step: 35

step: 36

step: 37

step: 38

step: 39

step: 40

step: 41

step: 42

step: 43

step: 44

step: 45

step: 46

step: 47

step: 48

step: 49

step: 50

step: 51

step: 52

step: 53

step: 54

step: 55

step: 56

step: 57

step: 58

step: 59

step: 60

step: 61

step: 62

step: 63

step: 64

step: 65

step: 66

step: 67

step: 68

step: 69

step: 70

step: 71

step: 72

step: 73

step: 74

step: 75

step: 76

step: 77

step: 78

step: 79

step: 80

step: 81

step: 82

step: 83

step: 84

step: 85

step: 86

step: 87

step: 88

step: 89

step: 90

step: 91

step: 92

step: 93

step: 94

step: 95

step: 96

step: 97

step: 98

step: 99

step: 100

,Round,Role,Gold,Killed_Wumpus,Map_Progress
0,1,HUNTER,0.000,0.0,1.583333
1,1,CARTOGRAPHER,0.000,0.0,1.950000
2,1,KNIGHT,0.000,0.0,2.066667
3,1,BWL_STUDENT,0.000,0.0,1.325000
4,2,HUNTER,0.000,0.0,2.633333
...,...,...,...,...,...
1191,298,BWL_STUDENT,0.575,0.0,56.300000
1192,299,HUNTER,1.000,1.3,96.600000
1193,299,CARTOGRAPHER,0.900,0.0,82.550000
1194,299,KNIGHT,1.700,3.4,138.166667


In [70]:
# Plot progress multiple run avg
colors = ["green", "violet", "red", "gold"]
legent = True
plt = make_subplots(rows=2, cols=2)
d = {}
goals = ["Gold", "Killed_Wumpus", "Map_Progress"]
for goalID in range(len(goals)):
    for role in AgentRole:
        df = data[data["Role"] == role.name]
        d["x"] = df["Round"].values
        d["y"] = df[goals[goalID]].values
        txt = [f"Role: {role.name}" for _ in range(0, number_of_steps)]
        plt.add_trace(go.Line(
            d, name=role.name, text=txt, showlegend = legent, line_color=colors[role.value]
        ), row=(goalID//2)+1, col=goalID%2+1)
    legent = False
    plt.update_xaxes(title_text="Round", row=(goalID//2)+1, col=goalID%2+1)
    plt.update_yaxes(title_text=goals[goalID], row=(goalID//2)+1, col=goalID%2+1)

plt

C:\Users\Luza3\PycharmProjects\Wumpus\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


